# Introduction

In [3]:
# Why is delineation of tumours important?
# How is delineation done today?
# What uncertainties are involved?
# CNNs for auto-delineation of tumours
# Propose solution - goal

# Theory

## Convolution Neural Network

In [4]:
# Define CNN here
# How effective
# Success story

# Software

## Requirements

Software requirement here, what will be included and not

### User Requirment Specification

What is a user? WHat can the user do?

### System Requirement Specification

Usability, Maintainability, hardware requirement, used packages

### Use cases

#### Use case diagram

Draw diagram here

#### Use case <1>

Explain use case here

## Designs

### Overview

#### Coding convention
PEP8, pydoc

#### Structure
Draw the main structure here

### Model

Explain what it does, then expose the api

### Infrastructure

#### Sequential

#### Unet

#### Dense

### Data

### Configuration

# Results

## Run on test data

# Discussions

Do another thing here

**Reference List**

- M Jameson, L Holloway, P Vial, S Vinod, P Metcalfe. 2010. A review of methods of analysis in contouring studies for radiation oncology. Journal of Medical Imaging and Radiation Oncology 54:401-410. Note: Understand the basics for your introduction:

"There is no consistent or widely accepted method of
systematic contour comparison. A number of contouring
metrics exist; some of which are available in treatment
planning systems and others require specialised software. "

"Volume was the most frequently used metric
across all tumour sites. Shape/dimension was the next
most frequently used metric in all tumour sites except for
brain and head and neck, where COV and CI were the
next most frequent metrics, respectively"

"the choice of metrics in many studies is
somewhat arbitrary and not determined on any established clinical basis. Further studies are needed to assess
the advantages and disadvantages of each metric in
various situations"

- C Njeh. 2008. Tumor delineation: The weakest link in the search for accuracy in radiotherapy. Journal of Medical Physics 33:136-140. Note: Understand the basics for your introduction [link](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2772050/)



In [1]:
import h5py

In [4]:
f = h5py.File('/NMBU/Thesis/full_dataset_singleclass.h5', 'r')

In [13]:
len(f)

3

In [24]:
for key in f.keys():
    print(key, len(f[key]['target']))

test 2793
train 9744
val 1033


In [22]:
len(f['train']['input'])

9744

In [26]:
int(13/3)

4

In [28]:
int(13/3.0)

4